<a href="https://colab.research.google.com/github/Decha117/AutoML/blob/main/AutoML_AutoGluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gdown
import gdown
gdown.download_folder("https://drive.google.com/drive/folders/1zUbkl_9tvSQXlq5yTKIQGDTmG65dgIJA?usp=share_link", quiet=True, use_cookies=False)

['/content/hearth-disease-recognition/test.csv',
 '/content/hearth-disease-recognition/sample_submission.csv',
 '/content/hearth-disease-recognition/train.csv']

# Preprocessing

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/hearth-disease-recognition/train.csv')
df_clean = df
col_del = ['ID', 'Income Level']
df_clean = df_clean.drop(col_del, axis=1)
df_clean = df_clean.dropna()

df_clean1 = df_clean
col_fac = ['History of HeartDisease or Attack', 'High Blood Pressure', 'Told High Cholesterol', 'Cholesterol Checked', 'Smoked 100+ Cigarettes', 'Diagnosed Stroke', 'Diagnosed Diabetes', 'Leisure Physical Activity', 'Heavy Alcohol Consumption', 'Health Care Coverage', 'Doctor Visit Cost Barrier', 'General Health', 'Difficulty Walking', 'Sex', 'Education Level', 'Vegetable or Fruit Intake (1+ per Day)']
for col in col_fac:
    df_clean1[col] = pd.factorize(df_clean[col])[0]

/tmp/ipython-input-2-1404900946.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean1[col] = pd.factorize(df_clean[col])[0]
/tmp/ipython-input-2-1404900946.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean1[col] = pd.factorize(df_clean[col])[0]
/tmp/ipython-input-2-1404900946.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [3]:
df_clean1.head(3)

,History of HeartDisease or Attack,High Blood Pressure,Told High Cholesterol,Cholesterol Checked,Body Mass Index,Smoked 100+ Cigarettes,Diagnosed Stroke,Diagnosed Diabetes,Leisure Physical Activity,Heavy Alcohol Consumption,Health Care Coverage,Doctor Visit Cost Barrier,General Health,Difficulty Walking,Sex,Education Level,Age,Vegetable or Fruit Intake (1+ per Day)
0,0,0,0,0,40.68,0,0,0,0,0,0,0,0,0,0,0,64,0
1,0,1,1,1,24.36,0,0,0,1,0,1,1,1,1,0,1,50,1
2,0,0,0,0,27.33,1,0,0,0,0,0,1,0,0,0,0,61,0


In [4]:
df_clean1['History of HeartDisease or Attack'].value_counts()

,count
History of HeartDisease or Attack,
0,162381
1,16938


# Split Label (y) / Features (X)

In [5]:
X = df_clean1.drop(['History of HeartDisease or Attack'], axis = 1)
y = df_clean1['History of HeartDisease or Attack']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, stratify=y, random_state = 0)

# Imbalanced check

In [6]:
print("Test "+str([sum(y_test==item) for item in np.unique(y_test)]))
print("Train "+str([sum(y_train==item) for item in np.unique(y_train)]))

Test [32476, 3388]
Train [129905, 13550]


# Oversampling

In [7]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
X, y = sm.fit_resample(X, y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, stratify=y, random_state = 0)

print("Test "+str([sum(y_test==item) for item in np.unique(y_test)]))
print("Train "+str([sum(y_train==item) for item in np.unique(y_train)]))

Test [32477, 32476]
Train [129904, 129905]


# Normalization

In [8]:
from sklearn.preprocessing import MinMaxScaler
scl = MinMaxScaler().fit(X_train)
X_train_norm = scl.transform(X_train)
X_test_norm = scl.transform(X_test)

# AutoML

In [9]:
!pip install -q autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.5/454.5 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.4/382.4 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 18.4 MB/s eta 0:0

In [10]:
from autogluon.tabular import TabularDataset, TabularPredictor

# AutoGluon works best directly with pandas DataFrames.
# Convert numpy arrays back to pandas DataFrames
X_train_norm_df = pd.DataFrame(X_train_norm, columns=X_train.columns)
X_test_norm_df = pd.DataFrame(X_test_norm, columns=X_test.columns)

# Combine features and target for AutoGluon training data
train_data = X_train_norm_df.copy()
train_data[y_train.name] = y_train.values

# Define the target column
label = y_train.name

# Initialize the predictor
# You can specify a directory to save models and results
predictor = TabularPredictor(label=label, path="AutogluonModels").fit(train_data)

# Evaluate the predictor on the test data
# Combine features and target for AutoGluon test data
test_data = X_test_norm_df.copy()
test_data[y_test.name] = y_test.values

performance = predictor.evaluate(test_data)
print(performance)

# To see the leaderboard of different models trained:
leaderboard = predictor.leaderboard(test_data, silent=True)
leaderboard

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          12
Memory Avail:       50.89 GB / 52.96 GB (96.1%)
Disk Space Avail:   192.88 GB / 235.68 GB (81.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='high'         : Strong accuracy with fast inference

[1000]	valid_set's binary_error: 0.166603
[2000]	valid_set's binary_error: 0.158523
[3000]	valid_set's binary_error: 0.152366
[4000]	valid_set's binary_error: 0.14698
[5000]	valid_set's binary_error: 0.143517
[6000]	valid_set's binary_error: 0.141208
[7000]	valid_set's binary_error: 0.139669
[8000]	valid_set's binary_error: 0.137745
[9000]	valid_set's binary_error: 0.13813
[10000]	valid_set's binary_error: 0.133513


	0.8665	 = Validation score   (accuracy)
	41.96s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8796	 = Validation score   (accuracy)
	15.69s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.875	 = Validation score   (accuracy)
	16.91s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: CatBoost ...
	0.8322	 = Validation score   (accuracy)
	41.96s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8619	 = Validation score   (accuracy)
	10.16s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8599	 = Validation score   (accuracy)
	10.88s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.8295	 = Validation score   (accuracy)
	139.75s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	0.8384	 = Validation score   (accuracy)
	4.74s	 = Training   runtime
	0

{'accuracy': 0.8892275953381676, 'balanced_accuracy': np.float64(0.8892282303127286), 'mcc': np.float64(0.7811176250721834), 'roc_auc': np.float64(0.9560814733193107), 'f1': 0.8936138752938741, 'precision': 0.859564784525672, 'recall': 0.9304717329720409}


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.889228,0.884956,accuracy,8.086601,0.472730,75.108725,0.005800,0.000938,0.247737,2,True,14
1,RandomForestGini,0.885563,0.879569,accuracy,1.008060,0.109982,15.691876,1.008060,0.109982,15.691876,1,True,5
2,RandomForestEntr,0.884070,0.874952,accuracy,1.338739,0.108836,16.913902,1.338739,0.108836,16.913902,1,True,6
3,LightGBM,0.877527,0.866487,accuracy,4.677665,0.202386,41.959612,4.677665,0.202386,41.959612,1,True,4
4,ExtraTreesGini,0.870152,0.861870,accuracy,1.119855,0.107867,10.160008,1.119855,0.107867,10.160008,1,True,8
5,ExtraTreesEntr,0.866242,0.859946,accuracy,1.115440,0.109522,10.878407,1.115440,0.109522,10.878407,1,True,9
6,NeuralNetTorch,0.852524,0.854175,accuracy,0.165143,0.013189,2848.844424,0.165143,0.013189,2848.844424,1,True,12
7,XGBoost,0.839576,0.838399,accuracy,0.332624,0.013132,4.735761,0.332624,0.013132,4.735761,1,True,11
8,LightGBMLarge,0.837575,0.836476,accuracy,0.256340,0.010555,3.985149,0.256340,0.010555,3.985149,1,True,13
9,CatBoost,0.835065,0.832243,accuracy,0.090133,0.003098,41.959944,0.090133,0.003098,41.959944,1,True,7


# Test


In [32]:
t = pd.read_csv('/content/hearth-disease-recognition/test.csv')
t.head()

,ID,High Blood Pressure,Told High Cholesterol,Cholesterol Checked,Body Mass Index,Smoked 100+ Cigarettes,Diagnosed Stroke,Diagnosed Diabetes,Leisure Physical Activity,Heavy Alcohol Consumption,Health Care Coverage,Doctor Visit Cost Barrier,General Health,Difficulty Walking,Sex,Education Level,Income Level,Age,Vegetable or Fruit Intake (1+ per Day)
0,test_000001,Yes,Yes,Yes,24.84,No,No,No,Yes,No,Yes,No,Good,No,Female,Some college or technical school,"$20,000 to less than $25,000",71,Yes
1,test_000002,Yes,No,Yes,29.08,Yes,No,No,No,No,Yes,No,Fair,No,Female,College graduate,"$50,000 to less than $75,000",61,No
2,test_000003,Yes,Yes,Yes,35.23,Yes,No,No,No,No,Yes,No,Fair,Yes,Female,Some college or technical school,"Less than $10,000",67,Yes
3,test_000004,No,No,Yes,24.78,Yes,No,No,No,No,Yes,No,Fair,No,Female,Some college or technical school,"$50,000 to less than $75,000",50,Yes
4,test_000005,No,No,Yes,27.57,Yes,No,No,No,No,Yes,No,Fair,No,Male,Some college or technical school,"$25,000 to less than $35,000",40,Yes


In [33]:
t.shape

(74361, 19)

In [34]:
col_del = ['ID', 'Income Level']
t = t.drop(col_del, axis=1)

t1 = t
col_fac = [
    'High Blood Pressure',
    'Told High Cholesterol',
    'Cholesterol Checked',
    'Smoked 100+ Cigarettes',
    'Diagnosed Stroke',
    'Diagnosed Diabetes',
    'Leisure Physical Activity',
    'Heavy Alcohol Consumption',
    'Health Care Coverage',
    'Doctor Visit Cost Barrier',
    'General Health',
    'Difficulty Walking',
    'Sex',
    'Education Level',
    'Vegetable or Fruit Intake (1+ per Day)'
]

for col in col_fac:
    t1[col] = pd.factorize(t[col])[0]

test = scl.transform(t1)

# Convert the numpy array to a pandas DataFrame before saving
test_df = pd.DataFrame(test, columns=t1.columns)
test_df.to_csv('test_norm.csv', index=False)
test_df.head()

,High Blood Pressure,Told High Cholesterol,Cholesterol Checked,Body Mass Index,Smoked 100+ Cigarettes,Diagnosed Stroke,Diagnosed Diabetes,Leisure Physical Activity,Heavy Alcohol Consumption,Health Care Coverage,Doctor Visit Cost Barrier,General Health,Difficulty Walking,Sex,Education Level,Age,Vegetable or Fruit Intake (1+ per Day)
0,0.0,0.0,0.0,0.155398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.646341,0.0
1,0.0,1.0,0.0,0.203988,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,0.0,0.0,0.2,0.524390,1.0
2,0.0,0.0,0.0,0.274467,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,1.0,0.0,0.0,0.597561,0.0
3,1.0,1.0,0.0,0.154710,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.390244,0.0
4,1.0,1.0,0.0,0.186683,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,0.0,1.0,0.0,0.268293,0.0


# Select Model

In [35]:
best_model_name = leaderboard.iloc[0]['model'] # WeightedEnsemble_L2

predictions = predictor.predict(test_df, model=best_model_name)
print(predictions)

0        1
1        0
2        0
3        0
4        0
        ..
74356    0
74357    1
74358    1
74359    0
74360    1
Name: History of HeartDisease or Attack, Length: 74361, dtype: int64


In [44]:
test_data_with_predictions = test_df.copy()
test_data_with_predictions['Predicted'] = predictions

# Save the dataframe with actuals and predictions to a CSV file
test_data_with_predictions.to_csv('test_predictions.csv', index=False)
submiss = pd.read_csv('test_predictions.csv')
submiss

,High Blood Pressure,Told High Cholesterol,Cholesterol Checked,Body Mass Index,Smoked 100+ Cigarettes,Diagnosed Stroke,Diagnosed Diabetes,Leisure Physical Activity,Heavy Alcohol Consumption,Health Care Coverage,Doctor Visit Cost Barrier,General Health,Difficulty Walking,Sex,Education Level,Age,Vegetable or Fruit Intake (1+ per Day),Predicted
0,0.0,0.0,0.0,0.155398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.646341,0.0,1
1,0.0,1.0,0.0,0.203988,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,0.0,0.0,0.2,0.524390,1.0,0
2,0.0,0.0,0.0,0.274467,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,1.0,0.0,0.0,0.597561,0.0,0
3,1.0,1.0,0.0,0.154710,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.390244,0.0,0
4,1.0,1.0,0.0,0.186683,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,0.0,1.0,0.0,0.268293,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74356,0.0,1.0,0.0,0.132592,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.353659,0.0,0
74357,0.0,0.0,0.0,0.194705,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.25,0.0,0.0,0.4,0.646341,0.0,1
74358,0.0,0.0,0.0,0.167316,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.75,1.0,0.0,0.2,0.878049,0.0,1
74359,0.0,0.0,0.0,0.142792,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.00,0.0,1.0,0.6,0.500000,1.0,0


In [45]:
cols_to_drop = [col for col in submiss.columns if col != 'Predicted']
submiss = submiss.drop(columns=cols_to_drop)
submiss

,Predicted
0,1
1,0
2,0
3,0
4,0
...,...
74356,0
74357,1
74358,1
74359,0


In [46]:
ids = pd.read_csv('/content/hearth-disease-recognition/sample_submission.csv')
ids = ids['ID']
ids

,ID
0,test_000001
1,test_000002
2,test_000003
3,test_000004
4,test_000005
...,...
74356,test_074357
74357,test_074358
74358,test_074359
74359,test_074360


In [47]:
submiss['ID'] = ids
submiss = submiss[['ID', 'Predicted']]
submiss.head()

,ID,Predicted
0,test_000001,1
1,test_000002,0
2,test_000003,0
3,test_000004,0
4,test_000005,0


In [48]:
submiss['Predicted'] = submiss['Predicted'].replace({1: 'Yes', 0: 'No'})
submiss.head()

,ID,Predicted
0,test_000001,No
1,test_000002,Yes
2,test_000003,Yes
3,test_000004,Yes
4,test_000005,Yes


In [49]:
submiss.to_csv('submission.csv', index=False)